In [9]:
from sklearn.metrics import classification_report, precision_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import xgboost as xgb

%store -r export_df
df = export_df
print(df)



x = df.drop(columns=['activity'])
y = df['activity']

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.1, random_state=42, stratify=y
)
y_train_encoded = label_encoder.transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

forest = RandomForestClassifier(n_estimators=200, random_state=42)
xgb_model = xgb.XGBClassifier(objective="multi:softprob", num_class=len(label_encoder.classes_), random_state=42)


forest.fit(x_train, y_train)
xgb_model.fit(x_train, y_train_encoded)

predictions = forest.predict(x_test)
predictions_xgb = xgb_model.predict(x_test)
predictions_xgb_decoded = label_encoder.inverse_transform(predictions_xgb)

acc = classification_report(y_test, predictions)
acc_xgb = classification_report(y_test, predictions_xgb_decoded)


print(acc_xgb)
print(df['activity'].value_counts())


            dxy      rsi_6     rsi_12       macd  macd_signal  macd_hist  \
49    97.300003  62.042073  57.750950  15.702564   -25.224870  40.927433   
50    97.300003  56.817640  55.176864  20.225745   -16.134747  36.360492   
51    97.300003  47.461901  50.396592  17.316035    -9.444590  26.760626   
52    97.300003  42.145623  47.528972  10.866365    -5.382399  16.248764   
53    97.300003  36.530449  44.378700   1.146351    -4.076649   5.223000   
...         ...        ...        ...        ...          ...        ...   
4314  97.778999  41.318937  41.531306 -51.897009   -59.615819   7.718811   
4315  97.778999  47.567674  44.318769 -49.048074   -57.502270   8.454196   
4316  97.778999  57.802929  49.321833 -42.984800   -54.598776  11.613976   
4317  97.778999  32.465955  37.180280 -48.646169   -53.408255   4.762086   
4318  97.778999  20.208374  28.388864 -65.098624   -55.746329  -9.352296   

             ema_21       sma_50     activity  
49    114975.319592  115072.3634   Stro